In [ ]:
!git clone https://github.com/serengil/deepface.git
%cd deepface
!pip install -e .

Cloning into 'deepface'...
remote: Enumerating objects: 8997, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 8997 (delta 242), reused 230 (delta 230), pack-reused 8451 (from 2)
Receiving objects: 100% (8997/8997), 55.20 MiB | 13.56 MiB/s, done.
Resolving deltas: 100% (5925/5925), done.
/content/deepface
Obtaining file:///content/deepface
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
  Running setup.py develop for deepface


In [ ]:
from google.colab import drive
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de tus archivos ZIP (modifica los nombres según los reales)
zips = [
    "/content/drive/MyDrive/TFG_Instagram/fotos_filtradas_LPGA.zip"
]

# Carpeta de destino base en /content
base_destino = "/content/"
os.makedirs(base_destino, exist_ok=True)

# Extraer cada ZIP en su propia subcarpeta
for zip_path in zips:
    nombre = os.path.splitext(os.path.basename(zip_path))[0]
    destino = os.path.join(base_destino, nombre)
    os.makedirs(destino, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print("Extraído")

Mounted at /content/drive
Extraído


In [ ]:
from PIL import Image
import os

# Ruta de la carpeta a analizar
carpeta = "/content/publicaciones_2024"  # cámbiala si es necesario

# Recorremos todos los archivos
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith((".jpg")):
        ruta = os.path.join(carpeta, archivo)
        try:
            with Image.open(ruta) as img:
                img.verify()  # Verifica si la imagen es válida
        except Exception as e:
            print(f"Imagen corrupta eliminada: {archivo} → {e}")
            os.remove(ruta)

Imagen corrupta eliminada: jsmunoz23_2024-10-05_00-27-16_UTC_3.jpg → cannot identify image file '/content/publicaciones_2024/jsmunoz23_2024-10-05_00-27-16_UTC_3.jpg'
Imagen corrupta eliminada: jsmunoz23_2024-09-23_14-59-13_UTC_2.jpg → cannot identify image file '/content/publicaciones_2024/jsmunoz23_2024-09-23_14-59-13_UTC_2.jpg'
Imagen corrupta eliminada: jsmunoz23_2024-09-22_21-29-02_UTC.jpg → cannot identify image file '/content/publicaciones_2024/jsmunoz23_2024-09-22_21-29-02_UTC.jpg'
Imagen corrupta eliminada: badds_2024-05-21_01-29-35_UTC_1.jpg → cannot identify image file '/content/publicaciones_2024/badds_2024-05-21_01-29-35_UTC_1.jpg'
Imagen corrupta eliminada: jsmunoz23_2024-10-05_00-27-16_UTC_2.jpg → cannot identify image file '/content/publicaciones_2024/jsmunoz23_2024-10-05_00-27-16_UTC_2.jpg'
Imagen corrupta eliminada: jsmunoz23_2024-10-02_23-10-14_UTC_1.jpg → cannot identify image file '/content/publicaciones_2024/jsmunoz23_2024-10-02_23-10-14_UTC_1.jpg'
Imagen corrupta 

In [ ]:
import os
import pandas as pd
from deepface import DeepFace
import gc
from tqdm import tqdm

directorio_imagenes = "/content/fotos_filtradas_LPGA"
csv_salida = "/content/drive/MyDrive/TFG_Instagram/emociones_por_imagen_final_LPGA.csv"

resultados = []

# Filtrar imágenes válidas
imagenes = sorted([
    f for f in os.listdir(directorio_imagenes)
    if f.lower().endswith("_utc.jpg") or f.lower().endswith("_utc_1.jpg")
])

for nombre_imagen in tqdm(imagenes, desc="Procesando imágenes", unit="img"):
    ruta_imagen = os.path.join(directorio_imagenes, nombre_imagen)
    nombre_salida = nombre_imagen.replace("_UTC_1.jpg", "_UTC.jpg")

    try:
        resultados_rostros = DeepFace.analyze(
            img_path=ruta_imagen,
            actions=["emotion"],
            detector_backend="retinaface",
            enforce_detection=True
        )

        if isinstance(resultados_rostros, dict):
            resultados_rostros = [resultados_rostros]

        contador = {}
        for r in resultados_rostros:
            emocion = r.get("dominant_emotion")
            if emocion:
                contador[emocion] = contador.get(emocion, 0) + 1

        if contador:
            emocion_predominante = max(contador, key=contador.get)
            total_rostros = len(resultados_rostros)
        else:
            emocion_predominante = "neutral"
            total_rostros = 0

    except Exception as e:
        emocion_predominante = "neutral"
        total_rostros = 0

    resultados.append((nombre_salida, total_rostros, emocion_predominante))
    gc.collect()

# Guardar resultados
df_resultados = pd.DataFrame(resultados, columns=["imagen", "Rostros", "Emoción"])
df_resultados.to_csv(csv_salida, index=False, sep=';')
print(f"Resultados guardados en: {csv_salida}")



25-07-26 12:29:27 - Directory /root/.deepface has been created
25-07-26 12:29:27 - Directory /root/.deepface/weights has been created


Procesando imágenes:   0%|          | 0/6679 [00:00<?, ?img/s]

25-07-26 12:29:31 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
  9%|▉         | 11.0M/119M [00:00<00:02, 38.5MB/s]
 18%|█▊        | 21.5M/119M [00:00<00:02, 38.0MB/s]
 27%|██▋       | 32.0M/119M [00:00<00:02, 34.6MB/s]
 36%|███▌      | 42.5M/119M [00:01<00:02, 35.3MB/s]
 39%|███▉      | 46.1M/119M [00:01<00:02, 26.7MB/s]
 42%|████▏     | 49.3M/119M [00:01<00:03, 21.4MB/s]
 44%|████▎     | 51.9M/119M [00:01<00:03, 19.2MB/s]
 46%|████▌     | 54.0M/119M [00:02<00:06, 10.2MB/s]
 53%|█████▎    | 63.4M/119M [00:02<00:03, 16.0MB/s]
 62%|██████▏   | 73.9M/119M [00:03<00:02, 22.1MB/s]
 71%|███████   | 84.4M/119M [00:03<00:01, 17.9MB/s]
 80%|███████▉  | 94.9M/119M [00:04<00:01, 22.2MB/s]
 89%|████████▉ | 105M/119M [00:04<00:00, 21.8MB/s] 
 91%|█████████ | 108M/119M [00:04<00:00, 22.0MB/s]
100%|██████████| 119M/119M [00:05<00:00, 23.0MB/s]


25-07-26 12:29:44 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 343MB/s]
Procesando imágenes: 100%|██████████| 6679/6679 [48:28<00:00,  2.30img/s]

Resultados guardados en: /content/drive/MyDrive/TFG_Instagram/emociones_por_imagen_final_LPGA.csv
